In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

In [ ]:
spark = (SparkSession
 .builder
 .appName("Profession")
 .getOrCreate())

Providing the Schema for the DataFrame.
Two Columns Salary and Profession

In [ ]:
schema = StructType([
             StructField('Profession', StringType(), True),
             StructField('Salary', StringType(), True)
         ])

In [ ]:
df =  spark.read.option('header','True').schema(schema).csv('German_Salaries_By_Profession.csv')

In [ ]:
df.printSchema()

In [ ]:
df.show()

In [ ]:
"""

The salary column had some currency symbols and commas needed to remove that so can change the datatype later easily

"""


df = df.withColumn('Salary',regexp_replace(df.Salary,pattern='[^a-zA-Z0-9]', replacement=''))

In [ ]:
"""

Creating row number for each of the professions

"""


window = Window.orderBy('Salary')
df = df.withColumn('Id',row_number().over(window))

df.show(truncate=False)

In [ ]:
"""

The file will be written to a Profession folder

"""

df.write.format('csv').option('header','true').mode('overwrite').save('Profession')